In [ ]:
def FICHA_INDICADOR(indicador, ruta):
    #Cargar info
    try:
        info = pd.read_excel(ruta, sheet_name="Registros", dtype=str)
    except FileNotFoundError:
        return {
            "info": None,
            "validacion": {"mensaje": "Archivo no encontrado"},
            "corresponde": None,
            "sentido": None,
            "comparacion_metodo_res": None
        }
    except Exception as e:
        return {
            "info": None,
            "validacion": {"mensaje": f"Error al leer el archivo: {e}"},
            "corresponde": None,
            "sentido": None,
            "comparacion_metodo_res": None
        }

    #Para que todas las metas se llamen igual
    if "META_REG1" in info.columns and "META1" not in info.columns:
        info.rename(columns={"META_REG1": "META1"}, inplace=True)

    #Columnas que se necesitan
    initial_cols = [
        "CICLO", "RAMO", "PP", "DESC_PROYECTO", "TIPO_NIVEL", "LLAVE_INDICADOR",
        "NOMBRE_INDICADOR", "DEFINICION_INDICADOR", "METODO_CALCULO_INDICADOR",
        "TIPO_RELATIVO", "FRECUENCIA_MEDICION", "DESC_SENTIDO", "UMBRAL_VERDE",
        "UMBRAL_ROJO", "LINEA_BASE", "CICLO_LINEA_BASE"
    ]
    meta_cols_pattern = r"META\d+"
    meta_cols = [col for col in info.columns if re.match(meta_cols_pattern, col)]

    cols_to_select = initial_cols + sorted(meta_cols)

    info = info[info["LLAVE_INDICADOR"] == str(indicador)][cols_to_select]

    #Si no hay info
    if info.empty:
        return {
            "info": None,
            "validacion": {"mensaje": "Indicador no encontrado"},
            "corresponde": None,
            "sentido": None,
            "comparacion_metodo_res": None
        }

    fila = info.iloc[0]

    #Ver si hay info
    no_info_parts = []
    if pd.isna(fila.get("NOMBRE_INDICADOR")): no_info_parts.append("Nombre del indicador no definido; ")
    if pd.isna(fila.get("DEFINICION_INDICADOR")): no_info_parts.append("Definición del indicador no definido; ")
    if pd.isna(fila.get("METODO_CALCULO_INDICADOR")): no_info_parts.append("Método de cálculo del indicador no definido; ")
    no_info = "".join(no_info_parts)

    #Para que coincida el tipo
    char_cols = [
        "PP", "DESC_PROYECTO", "TIPO_NIVEL", "NOMBRE_INDICADOR", "DEFINICION_INDICADOR",
        "METODO_CALCULO_INDICADOR", "TIPO_RELATIVO", "FRECUENCIA_MEDICION", "DESC_SENTIDO"
    ]

    for col in info.columns:
        if col in char_cols:
            info[col] = info[col].astype(str).replace('nan', None)
        else:
            info[col] = pd.to_numeric(info[col], errors='coerce')

    fila = info.iloc[0]

    if pd.isna(info['TIPO_RELATIVO'].iloc[0]):
      info['TIPO_RELATIVO'].iloc[0] = "Absoluto"

    #Si tiene frecuencia rara, ver si le toca o no revisión xd
    corresponde = "La frecuencia registrada no es bienal, trienal, quinquenal o sexenal"
    if not pd.isna(fila.get('CICLO')) and not pd.isna(fila.get('CICLO_LINEA_BASE')) and not pd.isna(fila.get('FRECUENCIA_MEDICION')):
        ciclo_diff = fila['CICLO'] - fila['CICLO_LINEA_BASE']
        if fila['FRECUENCIA_MEDICION'] == "Bienal":
            corresponde = "Corresponde" if ciclo_diff % 2 == 0 else "No corresponde"
        elif fila['FRECUENCIA_MEDICION'] == "Trienal":
            corresponde = "Corresponde" if ciclo_diff % 3 == 0 else "No corresponde"
        elif fila['FRECUENCIA_MEDICION'] == "Quinquenal":
            corresponde = "Corresponde" if ciclo_diff % 5 == 0 else "No corresponde"
        elif fila['FRECUENCIA_MEDICION'] == "Sexenal":
            corresponde = "Corresponde" if ciclo_diff % 6 == 0 else "No corresponde"

    #Si tiene sentido
    sentido = ""
    if corresponde != "No corresponde":
        if pd.isna(fila.get("DESC_SENTIDO")):
            sentido = "Sentido no definido"
        else:
            sentido = f"Sentido definido: {fila['DESC_SENTIDO']}"
    else:
        sentido = "No aplica (no corresponde medición)"

    #validar las metas según el snetido, si el sentido es vacío, entonces se evalúa como ascendente
    validacion = {"mensaje": ""}
    if corresponde == "Corresponde" or corresponde == "La frecuencia registrada no es bienal, trienal, quinquenal o sexenal":
        if pd.isna(fila.get("DESC_SENTIDO")) or fila["DESC_SENTIDO"] == "Ascendente":
            validacion = validar_metas_asc(info.copy())
        elif fila["DESC_SENTIDO"] == "Descendente":
            validacion = validar_metas_desc(info.copy())
        else:
            validacion = {"mensaje": "Error en el sentido registrado"}
    else:
        validacion = {"mensaje": "No corresponde medición"}

    #Comparar método de cálculo
    comparacion_metodo_res = {"mensaje": ""}
    if corresponde == "Corresponde" or corresponde == "La frecuencia registrada no es bienal, trienal, quinquenal o sexenal":
        comparacion_metodo_res = comparacion_indicador(info[['NOMBRE_INDICADOR', 'DEFINICION_INDICADOR','TIPO_RELATIVO', 'METODO_CALCULO_INDICADOR']].copy())
    else:
        comparacion_metodo_res = {"mensaje": "No corresponde medición"}

    return {
        "info": info,
        "no_info": no_info,
        "validacion": validacion,
        "corresponde": corresponde,
        "sentido": sentido,
        "comparacion_metodo_res": comparacion_metodo_res
    }